In [4]:
import re, os, sys
import pandas as pd

from config import KIEL_CORPUS_PATH
from data_structures.Interval import Interval

# Classes

In [82]:
class WordDurations:
    """
    Class to store phoneme info based on word, syllable position, etc
    """
    def __init__(self, word, segment = '', realization = '', basic = ''):
        self.word = str(word)
        self.segment = str(segment)
        self.realization = str(realization)
        self.basic = str(basic)
    def __str__(self):
        return f"Word: {self.word}\tsegment: {self.segment}\trealization: {self.realization}"
        

# Read Input

In [104]:
def extract_elements_from_line(master_list: list, line_list: list, line: str):
    """
    Extract elements from line into the master list
    """
    # element boundary is two white spaces for segment & realization tiers, one for original tier
    print(test)
    if '\n' in line_list:
        line_list.remove('\n')
    for i in range(len(line_list)):
        master_list.append(line_list[i])
    return master_list


In [105]:
def read_input(in_folder_name = KIEL_CORPUS_PATH):
    """
    read in all .s1h files from folder as list of WordDuration objects
    """
    intervals = []
    for file_name in os.listdir(in_folder_name):
        if not file_name.endswith('.s1h'):
            continue

        with open(os.path.join(in_folder_name, file_name), 'r', encoding='utf8') as in_file:
            is_orig_tier = False
            is_segment_tier = False
            is_realized_tier = False
            is_basic_tier = False
            
            original_tier = []
            segment_tier = []
            realized_tier = []
            basic_tier = []
            
            word_durations = []
            
            for i, line in enumerate(in_file):
                if i == 1:
                    is_orig_tier = True
                    orig_tier_list = line.strip('\n').split(' ')[2:]                    
                    extract_elements_from_line(original_tier, orig_tier_list, line)

                    original_tier = extract_elements_from_line(original_tier, orig_tier_list, line)

                        
                elif line.startswith('oend'):
                    is_orig_tier = False
                    is_segment_tier = True
                
                elif line.startswith('kend'):
                    is_segment_tier = False
                    is_realized_tier = True
                    
                    
                elif is_orig_tier:
                    orig_tier_list = line.strip('\n').split(' ')[8:]
                    original_tier = extract_elements_from_line(original_tier, orig_tier_list, line)

                        

                elif is_segment_tier:
                    segment_tier_list = line.strip('\n').split(' ' * 2)[1:]
                    segment_tier = extract_elements_from_line(segment_tier, segment_tier_list, line)
                
                elif is_realized_tier:
                    realized_tier_list = line.split(' ' * 2)[1:]
                    
                    realized_tier = extract_elements_from_line(realized_tier, realized_tier_list, line)

                        
        for item in list(zip(original_tier, segment_tier, realized_tier)):
            current_durations = WordDurations(word = item[0], segment = item[1], realization = item[2])
            word_durations.append(current_durations)
        
                    
#     for element in original_tier_words:
#         print(element)
    return word_durations

# output = read_input()
# for item in output:
#     print(item)
